### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import os
from matplotlib import pyplot as plt
from sklearn.linear_model import BayesianRidge, ARDRegression
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingRegressor

## Data Preprocessing

Load data

In [ ]:
DATA_PATH = "data/week-one/"
train_filename, test_filename, macro_filename = "X_train.csv", "X_test.csv", "macro.csv"

data = pd.read_csv(os.path.join(DATA_PATH, train_filename), parse_dates=['timestamp'])
test = pd.read_csv(os.path.join(DATA_PATH, test_filename), parse_dates=['timestamp'])
macro = pd.read_csv(os.path.join(DATA_PATH, macro_filename), parse_dates=['timestamp'])

In [ ]:
print(data.shape, test.shape, macro.shape)

Some useful functions used for the preprocessing of the data

In [ ]:
def plotCorrelation(data):
    corr_data = data.copy()
    names = list(corr_data.columns)
    correlations = corr_data.corr().abs()
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111)
    cax = ax.matshow(correlations, vmin=-1, vmax=1)
    # fig.colorbar(cax)
    ticks = np.arange(0,len(names),1)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(names)
    ax.set_yticklabels(names)
    plt.show()

def reduce(data, threshold=0.9):
    correlations = data.corr().abs()
    upper = correlations.where(
        np.triu(np.ones(correlations.shape), k=1).astype(np.bool))
    to_drop = [
        column for column in upper.columns if any(upper[column] > threshold)
    ]
    return data.drop(columns=to_drop)

def inpute(data, feature, verbose=False, **kwargs):
    X = data.copy().drop(columns=[feature])
    X = X.select_dtypes(exclude=['object'])
    X = X.fillna(X.median())
    y = data[feature]
    X_train = X[~y.isna()]
    X_test = X[y.isna()]
    y_train = y[~y.isna()]

    model = DecisionTreeRegressor(**kwargs)
    model.fit(X_train, y_train)
    if verbose:
        print("Feature: %s" % feature)
    filled_gaps = model.predict(X_test)
    for i, ind in enumerate(data[feature][data[feature].isna()].index):
        data.at[ind, feature] = filled_gaps[i]
    return data

Add macro data, only select features which are strongly correlated with the housing price. (This was compute offline). And extra feature, the year of sale is also added

In [ ]:
macro_features = ['timestamp', 'bandwidth_sports', 'fixed_basket', 'cpi', 'gdp_annual_growth',
                  'salary', 'deposits_value', 'load_of_teachers_school_per_teacher',
                  'turnover_catering_per_cap', 'gdp_deflator', 'gdp_annual']
plotCorrelation(macro)
macro = macro[macro_features]
# macro = macro.fillna(macro.median())
data = pd.merge_ordered(data, macro, on='timestamp', how='left')
data['year'] = pd.DatetimeIndex(data['timestamp']).year

Separate out target and features and exclude categorical features from training set

In [ ]:
y = data.copy()["price_doc"]
data.drop(['id', 'price_doc'], axis=1, inplace=True)
X = data.copy()

# Take only numeric data for now
X = X.select_dtypes(exclude=['object'])
X.drop(columns=["timestamp"], inplace=True)

Reduce dimensionality by removing strongly correlated features

In [ ]:
plotCorrelation(X)
X = reduce(X, threshold=0.95)
plotCorrelation(X)

#### Add categorical features using one-hot encoding

Some features would be best described with ordinal encoding

In [ ]:
data['ecology'] = data['ecology'].map({'excellent':4,'good':3,'satisfactory':2,'poor':1,'no data':np.nan})
X['ecology'] = data['ecology']

In [ ]:
for column in data.select_dtypes(include=['object']).drop(columns=['sub_area', 'product_type']).columns:
    data[column] = data[column].map({'yes':1, 'no':0})
    X[column] = data[column]

In [ ]:
X = pd.concat([X,pd.get_dummies(data.select_dtypes(include=['object']))], axis=1)

In [ ]:
print("Data shape:", X.shape)

Use a basic decision tree regressor to predict missing values in the data

In [ ]:
for column in X.columns[X.isna().any() == True]:
    X = inpute(X, column, min_samples_leaf=100)

In [ ]:
print("Data shape:", X.shape)

Remove outliers. Note: removing these values seemed to strongly reduce the accuracy of the model and in places gave infinities for the predictions. For these reasons, the outliers were kept in the dataset.

In [ ]:
z = pd.DataFrame(dict([(column,abs(stats.zscore(X[column]))) for column in X.columns]))

In [ ]:
# X = X[~((z > 5).sum(axis=1) > 5)]
print("Data shape:", X.shape)

## Model Selection

In [ ]:
# from weekone_models import models

models = {
    "ridge": {
        'model': sklearn.linear_model.Ridge(),
        'param_grid': {
            'ridge__alpha': np.logspace(2, 6, 10)
        }
    },
    "lasso": {
        'model': sklearn.linear_model.Lasso(),
        'param_grid': {
            'lasso__alpha': np.logspace(-5, 1, 10)
        }
    },
    "elasticnet": {
        'model': sklearn.linear_model.ElasticNet(),
        'param_grid': {
            'elasticnet__alpha': np.logspace(-5, 1, 10)
        }
    },
    "linearsvr": {
        'model': sklearn.svm.LinearSVR(),
        'param_grid': {
            'linearsvr__C': np.logspace(-5, 0, 5)
        }
    },
    "decisiontreeregressor": {
        'model': DecisionTreeRegressor(),
        'param_grid' : {
            'decisiontreeregressor__max_depth': np.logspace(0, 1.3, 10, dtype=int),
            'decisiontreeregressor__min_samples_leaf': np.logspace(2, 3, 5, dtype=int)
        }
    },
    "adaboostregressor": {
        'model': sklearn.ensemble.AdaBoostRegressor(DecisionTreeRegressor(max_depth=3)),
        'param_grid': {
            'adaboostregressor__n_estimators': np.logspace(0, 3, 10, dtype=int)
        }
    },
    "mlpregressor": {
        'model': MLPRegressor(),
        'param_grid': {
            'mlpregressor__alpha': np.logspace(-5,-1,10)
        }
    },
    "randomforestregressor": {
        'model': sklearn.ensemble.RandomForestRegressor(),
        'param_grid': {
            'n_estimators': np.logspace(1, 3, 20)
        }
    }
}

Iterate over each model defined in the dictionary and find optimal hyperparameters through RandomizedSearchCV (Previously GridSearchCV).

In [ ]:
for model in models:
    print("Performing search for %s model" % model)
    pipeline = make_pipeline(StandardScaler(), models[model]['model'])

    param_grid = models[model]['param_grid']

    gscv = RandomizedSearchCV(
        pipeline, param_grid, n_jobs=-1,
        scoring='neg_root_mean_squared_error', verbose=1, cv=5,
        refit='best_index_', n_iter=20
    )
    gscv.fit(X, np.log1p(y.loc[X.index]))
    models[model]['best_estimator'] = gscv.best_estimator_
    models[model]['best_score'] = gscv.best_score_
models

## Combine Models

Use a voting regressor to combine all of the models into an ensemble. Each model is weighted by its accuracy during the optimisation process. This ensemble is then fitted over the complete data set.

In [ ]:
model = make_pipeline(
    StandardScaler(),
    VotingRegressor(
        estimators=[(model, models[model]['best_estimator'].steps[1][1]) for model in models] + [],
        weights=[1/abs(models[model]['best_score']) for model in models],
        n_jobs=-1
    )
)

In [ ]:
model.fit(X,np.log1p(y.loc[X.index]))

In [ ]:
model.steps[1][1].estimators_

In [ ]:
mean_squared_error(
    model.predict(X),
    np.log1p(y.loc[X.index]),
    squared=False
)

## Run predictions on test data

In [ ]:
test['ecology'] = test['ecology'].map({'excellent':4,'good':3,'satisfactory':2,'poor':1,'no data':np.nan})
test = pd.merge_ordered(test, macro.fillna(macro.median()), on='timestamp', how='left')
for column in test.select_dtypes(include=['object']).drop(columns=['sub_area', 'product_type']).columns:
    test[column] = test[column].map({'yes':1, 'no':0})
X_predict = pd.concat([test.copy(),pd.get_dummies(test.select_dtypes(include=['object']))], axis=1)
for column in X.columns:
    if column not in X_predict:
        X_predict[column] = 0
X_predict = X_predict[X.columns]
for column in X_predict.columns[X_predict.isna().any() == True]:
    X_predict = inpute(X_predict, column, min_samples_leaf=100)

In [ ]:
predictions = np.expm1(model.predict(X_predict))
predictions = pd.DataFrame(predictions, columns=["price_doc"])
predictions = pd.concat([test['id'], predictions], axis=1)

In [ ]:
predictions.to_csv(os.path.join(DATA_PATH, "predictions.csv"), index=False)

## Using the GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gscv = GradientBoostingRegressor(), 
        {
            'max_depth': np.linspace(10, 30, 10, dtype=int),
            'min_samples_leaf': np.linspace(100, 200, 10, dtype=int),
            'n_estimators': np.linspace(300,400, 10, dtype=int),
            'learning_rate': np.linspace(0.01, 0.03, 10),
        }, 
        n_jobs=-1, n_iter=5, verbose=2, cv=5,
        scoring='neg_root_mean_squared_error',
        refit='best_index_'
)

In [ ]:
gscv.fit(X,np.log1p(y.loc[X.index]))

In [ ]:
gscv.best_score_

In [ ]:
gscv.best_estimator_

Best estimator copied into the code so as to reduce the number of repeats of the CV.

In [ ]:
best_estimator = sklearn.ensemble.GradientBoostingRegressor(
    alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
    init=None, learning_rate=0.018888888888888886,
    loss='ls', max_depth=16, max_features=None,
    max_leaf_nodes=None, min_impurity_decrease=0.0,
    min_impurity_split=None, min_samples_leaf=188,
    min_samples_split=2, min_weight_fraction_leaf=0.0,
    n_estimators=400, n_iter_no_change=None,
    presort='deprecated', random_state=None,
    subsample=1.0, tol=0.0001, validation_fraction=0.1,
    verbose=0, warm_start=False)

In [ ]:
best_estimator.fit(X,np.log1p(y.loc[X.index]))

MSE using the GradientBoostedRegressor is significantly lower than the ensemble predictions:

In [ ]:
mean_squared_error(
    best_estimator.predict(X),
    np.log1p(y.loc[X.index]),
    squared=False
)

In [ ]:
predictions = np.expm1(gscv.best_estimator_.predict(X_predict))
predictions = pd.DataFrame(predictions, columns=["price_doc"])
predictions = pd.concat([test['id'], predictions], axis=1)

In [ ]:
predictions.to_csv(os.path.join(DATA_PATH, "predictions.csv"), index=False)

### Feature importance

In [ ]:
model = sklearn.ensemble.RandomForestRegressor(max_depth=20, n_jobs=-1)


In [ ]:
model.fit(X,np.log1p(y.loc[X.index]))

In [ ]:
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=False).iloc[:10].plot.bar()
plt.ylabel("Feature importance");

Some notes:
- Adding PCA to the pipeline did not improve accuracy. Decision trees do not benefit from such transformations so this step was redundant and just increased compute time.
- Similarly, adding the macro data in its complete form served only increase the complexity fo the model, without improving the accuracy of the predictions.